In [1]:
# import torch
# from torch_geometric.data import Data, Dataset
# from torch.utils.data import DataLoader
# from torchvision import transforms
# from PIL import Image
# import os
# import numpy as np

# # Assuming you have a list of image file paths and corresponding labels
# class GraphImageDataset(Dataset):
#     def __init__(self, image_paths, labels, transform=None):
#         """
#         Custom dataset for loading graph data from images.
#         image_paths: List of paths to the images.
#         labels: List of labels for each image (node-level or graph-level).
#         transform: Optional transform to apply to the images.
#         """
#         self.image_paths = image_paths
#         self.labels = labels
#         self.transform = transform
        
#         # Define edge connections for the graph
#         self.connections = [
#             # Example of edge connections (based on facial landmarks, modify as needed)
#             (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8),
#             (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16),
#             (17, 18), (18, 19), (19, 20), (20, 21),
#             (22, 23), (23, 24), (24, 25), (25, 26),
#             (27, 28), (28, 29), (29, 30),
#             (30, 31), (31, 32), (32, 33), (33, 34), (34, 35),
#             (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 36),
#             (42, 43), (43, 44), (44, 45), (45, 46), (46, 47), (47, 42),
#             (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54),
#             (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60),
#             (60, 61), (61, 62), (62, 63), (63, 64), (64, 65), (65, 66), (66, 67), (67, 48),
#             (60, 67), (61, 66), (62, 65), (63, 64)
#         ]

#     def __len__(self):
#         return len(self.image_paths)

#     def __getitem__(self, idx):
#         # Load the image
#         image_path = self.image_paths[idx]
#         image = Image.open(image_path).convert("RGB")
        
#         # Apply transformations (resize, normalize, etc.)
#         if self.transform:
#             image = self.transform(image)
        
#         # Here, we assume node features are derived from the image (for example, facial landmark coordinates)
#         # For simplicity, assume that `image` is used to extract the node features (e.g., landmarks)
#         # You would likely use a pre-trained model or a facial landmark detector here
#         node_features = self.extract_node_features(image)
        
#         # Define the edge index (you can adjust the size according to your graph's structure)
#         edge_index = torch.tensor(self.connections, dtype=torch.long).t().contiguous()
        
#         # Get the label (assuming a graph-level classification task here)
#         label = self.labels[idx]
        
#         # Create a Data object to store the graph (nodes and edges)
#         data = Data(x=node_features, edge_index=edge_index, y=label)
        
#         return data
    
#     def extract_node_features(self, image):
#         """
#         A function to extract node features from the image. Here you would typically use a facial landmark detection method
#         to obtain the coordinates of facial landmarks (e.g., eyes, nose, etc.). 
#         For simplicity, we assume random node features here.
#         """
#         # For now, generating random node features (68 landmarks, 2D coordinates)
#         # You should replace this with actual facial landmark detection
#         node_features = torch.randn(68, 2)  # Example: 68 landmarks, 2D coordinates
        
#         return node_features

# # Define image transformations (resize, normalization, etc.)
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),  # Resize image
#     transforms.ToTensor(),  # Convert to tensor
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize
# ])

# # # Example usage:
# # # List of image paths and corresponding labels
# # image_paths = ["path_to_image1.jpg", "path_to_image2.jpg", "path_to_image3.jpg"]
# # labels = [0, 1, 2]  # Example labels (could be node-level or graph-level)

# # # Create the dataset
# # dataset = GraphImageDataset(image_paths=image_paths, labels=labels, transform=transform)

# # # Create DataLoader for batching
# # data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# # # Example loop through the DataLoader
# # for data in data_loader:
# #     # data is a batch of graphs
# #     # data.x = node features
# #     # data.edge_index = edge index
# #     # data.y = labels
# #     print(f"Node features: {data.x}")
# #     print(f"Edge index: {data.edge_index}")
# #     print(f"Labels: {data.y}")


In [2]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch_geometric.nn import GCNConv, TopKPooling, GraphSAGE

# class GCNN(nn.Module):
#     def __init__(self, num_nodes=68, num_features=2, num_classes=4):
#         super(GCNN, self).__init__()
        
#         # First Graph Convolutional Layer
#         self.conv1 = GCNConv(num_features, 64)
#         # Second Graph Convolutional Layer
#         self.conv2 = GCNConv(64, 64)
#         # Third Graph Convolutional Layer
#         self.conv3 = GCNConv(64, 64)
        
#         # Pooling layer to reduce the graph size (optional)
#         # Here we use TopKPooling, but you can use other pooling methods like GraphSAGEPooling, etc.
#         self.pool1 = TopKPooling(64, ratio=0.5)
        
#         # Fourth Graph Convolutional Layer
#         self.conv4 = GCNConv(64, 128)
#         # Fifth Graph Convolutional Layer
#         self.conv5 = GCNConv(128, 128)
        
#         # Second pooling layer
#         self.pool2 = TopKPooling(128, ratio=0.5)
        
#         # Fully Connected layers for classification
#         self.fc1 = nn.Linear(128 * num_nodes, 256)
#         self.fc2 = nn.Linear(256, num_classes)

#     def forward(self, x, edge_index, batch=None):
#         # First set of Graph Convolutional Layers
#         x = F.relu(self.conv1(x, edge_index))
#         x = F.relu(self.conv2(x, edge_index))
#         x = F.relu(self.conv3(x, edge_index))
        
#         # Pooling Layer
#         x, edge_index, _, batch, _ = self.pool1(x, edge_index, None, batch)
        
#         # Second set of Graph Convolutional Layers
#         x = F.relu(self.conv4(x, edge_index))
#         x = F.relu(self.conv5(x, edge_index))
        
#         # Pooling Layer
#         x, edge_index, _, batch, _ = self.pool2(x, edge_index, None, batch)
        
#         # Flatten the output for the fully connected layers
#         x = x.view(x.size(0), -1)
        
#         # Fully Connected layers
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)  # Output layer for classification
        
#         return x

# # Example usage of the model
# if __name__ == '__main__':
#     # Create a sample graph with 68 nodes and 2 features per node
#     num_nodes = 68
#     num_features = 2
#     num_classes = 7
    
#     # Random node features (num_nodes x num_features)
#     x = torch.randn(num_nodes, num_features)
    
#     # Define the edge connections
#     connections = [
#         # Jawline
#         (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8),
#         (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16),
#         # Right eyebrow
#         (17, 18), (18, 19), (19, 20), (20, 21),
#         # Left eyebrow
#         (22, 23), (23, 24), (24, 25), (25, 26),
#         # Nose bridge
#         (27, 28), (28, 29), (29, 30),
#         # Nose base
#         (30, 31), (31, 32), (32, 33), (33, 34), (34, 35),
#         # Right eye
#         (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 36),
#         # Left eye
#         (42, 43), (43, 44), (44, 45), (45, 46), (46, 47), (47, 42),
#         # Outer lip
#         (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54),
#         (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60),
#         (60, 61), (61, 62), (62, 63), (63, 64), (64, 65), (65, 66), (66, 67), (67, 48),
#         # Inner lip
#         (60, 67), (61, 66), (62, 65), (63, 64)
#     ]
    
#     # Convert edge connections into a tensor (edge_index)
#     x = torch.randn(68, 2)  # 68 nodes, each with 2 features

#     # Labels for classification (assuming 68 nodes with 4 classes for simplicity)
#     y = torch.randint(0, 4, (68,))  # Random labels for each node
    
#     # Create the graph data object (graph representation)
#     data = Data(x=x, edge_index=edge_index, y=y)
    
#     edge_index = torch.tensor(connections, dtype=torch.long).t().contiguous()
    
#     # Batch information (assuming only 1 graph here)
#     batch = torch.zeros(num_nodes, dtype=torch.long)
    
#     # Create the model
#     model = GCNN(num_nodes=num_nodes, num_features=num_features, num_classes=num_classes)
    
#     # Perform a forward pass
#     output = model(x, edge_index, batch)
#     print(output.shape)  # Should print torch.Size([68, 4]) as we have 68 nodes and 4 output classes


# Start GraphNeuralNetwork Code

In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import ast
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [4]:
def construct_adjacency_matrix(landmarks):
    num_nodes = len(landmarks)
    A = np.zeros((num_nodes, num_nodes))

    connections = [
        # Jawline
        (0, 1),
        (1, 2),
        (2, 3),
        (3, 4),
        (4, 5),
        (5, 6),
        (6, 7),
        (7, 8),
        (9, 10),
        (10, 11),
        (11, 12),
        (12, 13),
        (13, 14),
        (14, 15),
        (15, 16),
        # Right eyebrow
        (17, 18),
        (18, 19),
        (19, 20),
        (20, 21),
        # Left eyebrow
        (22, 23),
        (23, 24),
        (24, 25),
        (25, 26),
        # Nose bridge
        (27, 28),
        (28, 29),
        (29, 30),
        # Nose base
        (30, 31),
        (31, 32),
        (32, 33),
        (33, 34),
        (34, 35),
        # Right eye
        (36, 37),
        (37, 38),
        (38, 39),
        (39, 40),
        (40, 41),
        (41, 36),
        # Left eye
        (42, 43),
        (43, 44),
        (44, 45),
        (45, 46),
        (46, 47),
        (47, 42),
        # Outer lip
        (48, 49),
        (49, 50),
        (50, 51),
        (51, 52),
        (52, 53),
        (53, 54),
        (54, 55),
        (55, 56),
        (56, 57),
        (57, 58),
        (58, 59),
        (59, 60),
        (60, 61),
        (61, 62),
        (62, 63),
        (63, 64),
        (64, 65),
        (65, 66),
        (66, 67),
        (67, 48),
        # Inner lip
        (60, 67),
        (61, 66),
        (62, 65),
        (63, 64),
    ]

    for i, j in connections:
        if i < num_nodes and j < num_nodes:
            A[i, j] = 1
            A[j, i] = 1

    return A

In [5]:
import numpy as np
from scipy.spatial import Delaunay

def delaunay_triangulation(landmarks):
    """
    Perform Delaunay triangulation and return triangles and edges.
    """
    tri = Delaunay(landmarks)
    triangles = tri.simplices
    edges = set()
    for triangle in triangles:
        for i in range(3):
            edge = tuple(sorted([triangle[i], triangle[(i + 1) % 3]]))
            edges.add(edge)
    return triangles, list(edges)

def construct_adjacency_matrix(landmarks):
    """
    Construct an adjacency matrix from the Delaunay triangulation of the landmarks.
    
    Args:
        landmarks (ndarray): Array of 2D points (n, 2) representing the landmarks.
    
    Returns:
        adjacency_matrix (ndarray): A binary adjacency matrix of shape (n, n).
    """
    _, edges = delaunay_triangulation(landmarks)
    num_landmarks = len(landmarks)
    adjacency_matrix = np.zeros((num_landmarks, num_landmarks), dtype=int)
    
    for edge in edges:
        i, j = edge
        adjacency_matrix[i, j] = 1
        adjacency_matrix[j, i] = 1  # Symmetric for undirected graph

    return adjacency_matrix

# Example usage
landmarks = np.array([[0, 0], [1, 0], [0, 1], [1, 1], [0.5, 0.5]])  # Sample landmarks
adjacency_matrix = construct_adjacency_matrix(landmarks)
print("Adjacency Matrix:")
print(adjacency_matrix)


Adjacency Matrix:
[[0 1 1 0 1]
 [1 0 0 1 1]
 [1 0 0 1 1]
 [0 1 1 0 1]
 [1 1 1 1 0]]


In [6]:
def master_node_adjacency_matrix(landmarks):
    num_landmarks = len(landmarks)
    master_node = 31
    adjacency_matrix = np.zeros((num_landmarks, num_landmarks), dtype=int)

    for i in range(num_landmarks):
        if i != master_node:
            adjacency_matrix[master_node, i] = 1
            adjacency_matrix[i, master_node] = 1

    return adjacency_matrix

In [7]:
def normalize_adjacency_matrix(A):
    A = A + np.eye(A.shape[0])  # Add self-connections
    D = np.diag(np.sum(A, axis=1))
    D_inv_sqrt = np.linalg.inv(np.sqrt(D))
    A_normalized = D_inv_sqrt @ A @ D_inv_sqrt
    return A_normalized

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling, GraphSAGE

# class GCNN(nn.Module):
#     def __init__(self, num_nodes=68, num_features=2, num_classes=4):
#         super(GCNN, self).__init__()
        
#         # First Graph Convolutional Layer
#         self.conv1 = GCNConv(num_features, 64)
#         # Second Graph Convolutional Layer
#         self.conv2 = GCNConv(64, 64)
#         # Third Graph Convolutional Layer
#         self.conv3 = GCNConv(64, 64)
        
#         # Pooling layer to reduce the graph size (optional)
#         # Here we use TopKPooling, but you can use other pooling methods like GraphSAGEPooling, etc.
#         self.pool1 = TopKPooling(64, ratio=0.5)
        
#         # Fourth Graph Convolutional Layer
#         self.conv4 = GCNConv(64, 128)
#         # Fifth Graph Convolutional Layer
#         self.conv5 = GCNConv(128, 128)
        
#         # Second pooling layer
#         self.pool2 = TopKPooling(128, ratio=0.5)
        
#         # Fully Connected layers for classification
#         self.fc1 = nn.Linear(128 * num_nodes, 256)
#         self.fc2 = nn.Linear(256, num_classes)

#     def forward(self, x, edge_index, batch=None):
#         # First set of Graph Convolutional Layers
#         x = F.relu(self.conv1(x, edge_index))
#         x = F.relu(self.conv2(x, edge_index))
#         x = F.relu(self.conv3(x, edge_index))
        
#         # Pooling Layer
#         x, edge_index, _, batch, _ = self.pool1(x, edge_index, None, batch)
        
#         # Second set of Graph Convolutional Layers
#         x = F.relu(self.conv4(x, edge_index))
#         x = F.relu(self.conv5(x, edge_index))
        
#         # Pooling Layer
#         x, edge_index, _, batch, _ = self.pool2(x, edge_index, None, batch)
        
#         # Flatten the output for the fully connected layers
#         x = x.view(x.size(0), -1)
        
#         # Fully Connected layers
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)  # Output layer for classification
        
#         return x

In [9]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch_geometric.nn import GCNConv, TopKPooling

# class GCNN(nn.Module):
#     def __init__(self, num_nodes=68, num_features=2, num_classes=7):
#         super(GCNN, self).__init__()
        
#         # First Graph Convolutional Layer
#         self.conv1 = GCNConv(num_features, 64)  # Input: (num_nodes, num_features) -> Output: (num_nodes, 64)
#         # Second Graph Convolutional Layer
#         self.conv2 = GCNConv(64, 64)             # Input: (num_nodes, 64) -> Output: (num_nodes, 64)
#         # Third Graph Convolutional Layer
#         self.conv3 = GCNConv(64, 64)             # Input: (num_nodes, 64) -> Output: (num_nodes, 64)
        
#         # Pooling layer to reduce the graph size
#         self.pool1 = TopKPooling(64, ratio=0.5)  # Input: (num_nodes, 64) -> Output: (num_pooled_nodes, 64)
        
#         # Fourth Graph Convolutional Layer
#         self.conv4 = GCNConv(64, 128)            # Input: (num_pooled_nodes, 64) -> Output: (num_pooled_nodes, 128)
#         # Fifth Graph Convolutional Layer
#         self.conv5 = GCNConv(128, 128)           # Input: (num_pooled_nodes, 128) -> Output: (num_pooled_nodes, 128)
        
#         # Second pooling layer
#         self.pool2 = TopKPooling(128, ratio=0.5) # Input: (num_pooled_nodes, 128) -> Output: (num_pooled_nodes2, 128)
        
#         # Fully Connected layers for classification
#         self.fc1 = None  # Initialize as None
#         self.fc2 = nn.Linear(256, num_classes)  # Output layer: Input: (batch_size, 256) -> Output: (batch_size, num_classes)

#     def forward(self, x, edge_index, batch=None):
#         # before applying convolutional layer the size of the x and edge index 
#         print(f"After conv1: x size = {x.size()}, edge_index size = {edge_index.size()}")
        
#         # First set of Graph Convolutional Layers
#         x = F.relu(self.conv1(x, edge_index))  # After conv1: x size = (num_nodes, 64)
#         print(f"After conv1: x size = {x.size()}, edge_index size = {edge_index.size()}")
        
#         x = F.relu(self.conv2(x, edge_index))  # After conv2: x size = (num_nodes, 64)
#         print(f"After conv2: x size = {x.size()}, edge_index size = {edge_index.size()}")
        
#         x = F.relu(self.conv3(x, edge_index))  # After conv3: x size = (num_nodes, 64)
#         print(f"After conv3: x size = {x.size()}, edge_index size = {edge_index.size()}")
        
#         # Pooling Layer
#         x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)  # After pool1: x size = (num_pooled_nodes, 64)
#         print(f"After pool1: x size = {x.size()}, edge_index size = {edge_index.size()}, batch size = {batch.size()}")
        
#         # Second set of Graph Convolutional Layers
#         x = F.relu(self.conv4(x, edge_index))  # After conv4: x size = (num_pooled_nodes, 128)
#         print(f"After conv4: x size = {x.size()}, edge_index size = {edge_index.size()}")
        
#         x = F.relu(self.conv5(x, edge_index))  # After conv5: x size = (num_pooled_nodes, 128)
#         print(f"After conv5: x size = {x.size()}, edge_index size = {edge_index.size()}")
        
#         # Pooling Layer
#         x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)  # After pool2: x size = (num_pooled_nodes2, 128)
#         print(f"After pool2: x size = {x.size()}, edge_index size = {edge_index.size()}, batch size = {batch.size()}")
        
#         # Flatten the output for the fully connected layers
#         x = x.view(x.size(0), -1)  # After flattening: x size = (batch_size, 128 * num_pooled_nodes2)
#         print(f"After flattening: x size = {x.size()}")
        
#         # Define fc1 based on the flattened size
#         if self.fc1 is None:
#             self.fc1 = nn.Linear(x.size(1), 256)  # Set input size dynamically: Input: (batch_size, 128 * num_pooled_nodes2) -> Output: (batch_size, 256)

#         # Fully Connected layers
#         x = F.relu(self.fc1(x))  # After fc1: x size = (batch_size, 256)
#         print(f"After fc1: x size = {x.size()}")
        
#         x = self.fc2(x)  # Output layer for classification: Input: (batch_size, 256) -> Output: (batch_size, num_classes)
#         print(f"After fc2: x size = {x.size()}")
        
#         return x

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling

class GCNN(nn.Module):
    def __init__(self, num_nodes=68, num_features=2, num_classes=7):
        super(GCNN, self).__init__()
        
        # First Graph Convolutional Layer
        self.conv1 = GCNConv(num_features, 64)
        self.conv2 = GCNConv(64, 64)
        self.conv3 = GCNConv(64, 64)
        
        # Pooling layer to reduce the graph size
        self.pool1 = TopKPooling(64, ratio=0.5)
        
        # Fourth Graph Convolutional Layer
        self.conv4 = GCNConv(64, 128)
        self.conv5 = GCNConv(128, 128)
        
        # Second pooling layer
        self.pool2 = TopKPooling(128, ratio=0.5)
        
        # Fully Connected layers for classification
        self.fc2 = nn.Linear(256, num_classes)  # Output layer: Input: (batch_size, 256) -> Output: (batch_size, num_classes)

    def forward(self, x, edge_index, batch=None):
        # First set of Graph Convolutional Layers
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = F.relu(self.conv3(x, edge_index))
        
        # Pooling Layer
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        
        # Second set of Graph Convolutional Layers
        x = F.relu(self.conv4(x, edge_index))
        x = F.relu(self.conv5(x, edge_index))
        
        # Pooling Layer
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        
        # Flatten the output for the fully connected layers
        x = x.view(x.size(0), -1)  # Flattening
        
        # Define fc1 based on the flattened size
        fc1_input_size = x.size(1)  # Get the actual input size dynamically
        self.fc1 = nn.Linear(fc1_input_size, 256)  # Set input size dynamically

        # Fully Connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [11]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from tqdm import tqdm
import ast

In [12]:
# Function to load data from CSV
def load_data_from_csv(csv_file):
    data = pd.read_csv(csv_file)
    
    images = []
    landmarks = []
    expressions = []
    
    for _, row in data.iterrows():
        image_name = row['image_name']
        
        # Convert string representation of list into actual list
        landmark_points = ast.literal_eval(row['landmark_points'])  # Convert string to list
        expression = np.array(ast.literal_eval(row['expressions']))  # Ensure it's a numpy array from string

        # Convert landmark points to numpy array
        landmarks.append(np.array(landmark_points))
        expressions.append(expression)
        images.append(image_name)
        
    return images, landmarks, expressions

In [13]:
# Convert landmarks and labels into PyTorch tensors
def create_graph_data(landmarks, expression, num_nodes=68):
    """
    Create a Data object for PyTorch Geometric.
    
    Args:
        landmarks (numpy.ndarray): Landmark points (shape: [num_landmarks, 2]).
        expression (numpy.ndarray): One-hot encoded expression (shape: [num_classes]).
        
    Returns:
        data (torch_geometric.data.Data): PyTorch Geometric Data object.
    """
    # Feature matrix (landmark points)
    x = torch.tensor(landmarks, dtype=torch.float)
    
    # Create adjacency matrix using the function defined earlier
    adjacency_matrix = construct_adjacency_matrix(landmarks)
    
    # Edge index (convert adjacency matrix to edge list)
    edge_index = torch.tensor(np.nonzero(adjacency_matrix), dtype=torch.long)
    
    # Label (expression one-hot encoded)
    y = torch.tensor(expression, dtype=torch.float)
    
    data = Data(x=x, edge_index=edge_index, y=y)
    
    return data

In [14]:
# Load your CSV data
train_images, train_landmarks, train_expressions = load_data_from_csv('tif_df_train_image.csv')  # Update with actual path
test_images, test_landmarks, test_expressions = load_data_from_csv('tif_df_test_image.csv')  # Update with actual path

In [15]:
# Create graph data for the dataset
train_dataset = [create_graph_data(l, exp) for l, exp in zip(train_landmarks, train_expressions)]
test_dataset = [create_graph_data(l, exp) for l, exp in zip(test_landmarks, test_expressions)]

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Model Initialization
model = GCNN(num_nodes=68, num_features=2, num_classes=7).to('cuda:1')  # Assuming GPU training

# Loss and Optimizer
criterion = torch.nn.BCEWithLogitsLoss()  # Binary Cross Entropy with logits for multi-class one-hot targets
optimizer = optim.Adam(model.parameters(), lr=0.001)

/tmp/ipykernel_73485/2487346700.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  edge_index = torch.tensor(np.nonzero(adjacency_matrix), dtype=torch.long)
/raid/home/dgx1575/20je0167_anurag/.venv/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [16]:
# Training Loop with Validation
def train_and_validate(model, train_loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for data in tqdm(train_loader, desc=f"Training Epoch {epoch}"):
        data = data.to(device)
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data.x, data.edge_index, batch=data.batch)
        
        # Compute loss
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Calculate accuracy
        predicted = (output > 0.5).float()  # Apply threshold for binary classification
        correct += (predicted == data.y).sum().item()
        total += data.y.size(0) * data.y.size(1)  # For multi-class (one-hot), multiply by number of classes
    
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch} - Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.2f}%")


In [17]:
# Set device
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Send the model to GPU if available
model = model.to(device)

# Train the model
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train_and_validate(model, train_loader, criterion, optimizer, epoch)

Training Epoch 1:   0%|                                                                                                              | 0/4 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:1! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)